In [ ]:
%matplotlib

import re
import copy
import sys
from enum import Enum
import matplotlib.pyplot as plt
import time
import numpy as np
import random

In [ ]:
instances_path = 'JSPLIB/instances/'
instance_path = instances_path + 'ft06'

def SPT(possible_tasks):
    smallest_duration = sys.maxsize
    best_task = None
    for t in possible_tasks:
        if t.duration < smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

def LPT(possible_tasks):
    smallest_duration = 0
    best_task = None
    for t in possible_tasks:
        if t.duration > smallest_duration:
            smallest_duration = t.duration
            best_task = t
    return best_task

class STATE(Enum):
    NOT_DONE = '--'
    ON_GOING = '+/-' 
    DONE = '++'
    
class Task:     
    def __init__(self,ressource,duration,job_num):
        self.res = ressource
        self.job_num = job_num
        self.duration = duration
        self.state = STATE.NOT_DONE
        
    def to_string(self):
        return str(self.res) + " " + str(self.duration)

    def start(self,time):
        self.start_time = time
        self.state = STATE.ON_GOING
    
    def update(self,time):
        if(self.state == STATE.ON_GOING):
            if time >= self.start_time + self.duration:
                self.state = STATE.DONE
                return self.res
        return -1
                
    def is_done(self):
        return self.state == STATE.DONE
        
class Job:
    def __init__(self,job_num):
        self.tasks = []
        self.num = job_num
        
    def add_task(self,task):
        self.tasks.append(task)
        
    def to_string(self):
        res = "|"
        for t in self.tasks:
            res = res + " " + '{:^6}'.format(t.to_string()) + " |" 
        return res
    
    def get_next_task(self):
        on_going = [task for task in self.tasks if task.state == STATE.ON_GOING]
        if  len(on_going) == 0:
            not_done_tasks = [task for task in self.tasks if task.state == STATE.NOT_DONE]
            if(len(not_done_tasks) != 0):
                return not_done_tasks[0]
            else:
                return None
        
    def start_next_task(self,time):
        t = next(task for task in self.tasks if task.state == STATE.NOT_DONE)
        if t != None:
            t.start(time)
            return t.res
        
    def update(self,time):
        res_to_free = []
        for t in self.tasks:
            res_to_free.append(t.update(time))
        return res_to_free
            
    def is_done(self):
        done = True
        for t in self.tasks:
            done = done & t.is_done()
        return done
        
class Instance:
    def __init__(self,n_machines,jobs):
        self.n_machines = n_machines   
        self.jobs=jobs
        self.machines_availability = [True]*n_machines
        self.time = 0

    def get_max_tasks(self):
        nb_max=0
        for j in self.jobs:
            if(len(j.tasks)>nb_max):
                nb_max = len(j.tasks)
        return nb_max
    
    def check_possible_tasks(self):
        possible_tasks = []
        for j in self.jobs:
            next_task = j.get_next_task()
            if(next_task != None):
                if(self.machines_availability[next_task.res]):
                    possible_tasks = possible_tasks + [next_task]
        return possible_tasks
    
    def display_jobs(self):
        max_tasks_per_job = 0
        for j in self.jobs:
            if(max_tasks_per_job < len(j.tasks)):
                max_tasks_per_job = len(j.tasks)
        header=" "
        for t in range(max_tasks_per_job):
            header = header + " " + '{:^6}'.format("R D") + "  " 
        print(header)
        for j in self.jobs:
            print(j.to_string())
            
    def get_state_matrix(self):
        m = np.zeros((len(self.jobs),self.get_max_tasks()))
        for j_num,j in enumerate(self.jobs):
            red_tasks=[]
            orange_tasks=[]
            green_tasks=[]
            for i,t in enumerate(j.tasks):
                if(t.state == STATE.ON_GOING):     
                    m[j_num][i] = 0             
                elif(t.state == STATE.DONE):
                    m[j_num][i] = 1           
                else:
                    m[j_num][i] = -1 
        return m
            
        
    def get_all_tasks(self):
        all_tasks = []
        for j in self.jobs:
            all_tasks = all_tasks + j.tasks
        return all_tasks
    
    def time_forward(self):
        self.time = self.time + 1
        for j in self.jobs:
            res_to_free = j.update(self.time)
            
            for r in res_to_free:
                if(r != -1):
                    self.machines_availability[r] = True
        
            
    def start_job(self,job_num):
        res_used = self.jobs[job_num].start_next_task(self.time)
        self.machines_availability[res_used] = False
        
    def is_done(self):
        done = True
        for j in self.jobs:
            done = done & j.is_done()
        return done
    
    def start_heuristic(self,fct=SPT,rand=0,display=False):
        job_order = []
        if(display):
            matrix_state = self.get_state_matrix()
            fig, ax = plt.subplots()
            grid_x=[i+0.5 for i in range(matrix_state.shape[1])]
            ax.set_xticks(grid_x,minor=True)
            grid_y=[i+0.5 for i in range(matrix_state.shape[0])]
            ax.set_yticks(grid_y,minor=True)
            ax.grid(which='minor')
            ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
            for i,jb in enumerate(self.jobs):
                for j,t in enumerate(jb.tasks):
                    text = ax.text(j, i, t.res,ha="center", va="center", color='black', fontsize=16)
            plt.xlabel('Tasks')
            plt.ylabel('Jobs')
            
        while not self.is_done():
            possible_tasks = self.check_possible_tasks()
            if(len(possible_tasks) > 0):
                if(random.random()<rand):
                    highest_prio_task = random.choice(possible_tasks)
                else:
                    highest_prio_task = fct(possible_tasks)
                self.start_job(highest_prio_task.job_num)
                job_order.append(highest_prio_task.job_num)
            else:
                self.time_forward()
                
                if(display):
                    matrix_state = self.get_state_matrix()
                    ax.matshow(matrix_state, cmap='RdYlGn', vmin=-1, vmax=1)
                    ax.set_title("TIME : "+str(self.time))
                    plt.pause(0.01)
                    
        return job_order
        
        


In [ ]:
list_jobs = []
n_machines = 0
n_jobs = 0
# Lecture de l'instance
with open(instance_path,'r') as f:
    num_line = 0
    for line in f:
        if(line[0] != '#'):
            
            re_line = re.sub(' +',';',line)
            re_line = re.sub('\\n','',re_line)
            split_line = re_line.split(';')
            if (num_line == 0):
                n_machines = int(split_line[1])
                n_jobs = int(split_line[0])
            else:
                list_jobs.append(split_line)
            num_line = num_line+1
list_jobs

In [ ]:
# Création des jobs à partir des listes précédentes
jobs = []
for i in range(n_machines):
    job = Job(i)
    for j in range(0,2*n_jobs,2):
        job.add_task(Task(int(list_jobs[i][j]),int(list_jobs[i][j+1]),i))
    jobs.append(job)
    
instance = Instance(n_machines,jobs)

In [ ]:


print("Nombre de machines : ",instance.n_machines)
print('Nombre de jobs : ',len(instance.jobs))
print('Taches : ')
instance.display_jobs()

In [ ]:
res = instance.start_heuristic(fct=LPT,rand=1,display=True)
print(res)

In [ ]:
print(instance.time)